In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Prepare Data set for Cats and Dogs:

'1. Data preprocessing: Image Generator, Train Test Split'
'2. Model architecting'
'3. Model compile'
'4. Model Traiing'
'5. Model Testing / Input testing'

'5. Model Testing / Input testing'

In [4]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

In [8]:
train_image_data = train_image_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/train',
                                                             class_mode= 'binary',
                                                             target_size = (64,64),
                                                             batch_size = 10
                                                             )

test_image_data = test_image_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/validation', class_mode='binary',
                                                           target_size = (64,64),
                                                           batch_size = 10)

Found 2000 images belonging to 2 classes.
Found 1002 images belonging to 2 classes.


In [ ]:

vgg = tf.keras.applications.vgg16.VGG16(include_top=False)
vgg.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
